In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
import pyspark.sql.functions
from pyspark.sql.functions import col,when

In [2]:
accident = pd.read_csv('/home/linuxu/Desktop/1.csv')
modifiedAccident = accident.dropna()

/home/linuxu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#filter
modifiedAccident1 = modifiedAccident[modifiedAccident['Road_Type'] >= 0]
modifiedAccident2 = modifiedAccident1[modifiedAccident1['Speed_limit'] >= 0]
modifiedAccident3 = modifiedAccident2[modifiedAccident2['Junction_Detail'] >= 0]
modifiedAccident4 = modifiedAccident3[modifiedAccident3['2nd_Road_Class'] >= 0]
modifiedAccident5 = modifiedAccident4[modifiedAccident4['Junction_Control'] >= 0]
modifiedAccident6 = modifiedAccident5[modifiedAccident5['Light_Conditions'] >= 0]
modifiedAccident7 = modifiedAccident6[modifiedAccident6['Weather_Conditions'] >= 0]
modifiedAccident8 = modifiedAccident7[modifiedAccident7['Road_Surface_Conditions'] >= 0]
modifiedAccident9 = modifiedAccident8[modifiedAccident8['Special_Conditions_at_Site'] >= 0]
modifiedAccident10 = modifiedAccident9[modifiedAccident9['1st_Road_Class'] >= 0]
#filter

In [4]:
subData = modifiedAccident[0:1200000]
subData.to_csv('/home/linuxu/Desktop/subData.csv',index=False)

In [5]:
allColumns = ['1st_Road_Class', 'Road_Type', 'Speed_limit', 'Junction_Detail', 'Junction_Control', '2nd_Road_Class', 'Light_Conditions', 'Weather_Conditions', 
              'Road_Surface_Conditions', 'Special_Conditions_at_Site']

In [7]:

i = 1
while i <= 1:
  spark = SparkSession.builder.appName('ml-bank').getOrCreate()
  df = spark.read.csv('/home/linuxu/Desktop/subData.csv', header = True, inferSchema = True)
  comb = random.sample(allColumns, k = 10)

  fc = open("/home/linuxu/Desktop/CombinationsTried.txt","a+")
  sentence = str(i)+ ".  "+ ', '.join(map(str, comb))  + "\n"
  fc.write(sentence)
  fc.close()

    #binary
  df = df.withColumn("Accident_Severity", when(col("Accident_Severity")  == 3,0).otherwise(1))

  df = df.select('Accident_Severity', comb[0], comb[1], comb[2], comb[3], comb[4], comb[5], comb[6], comb[7], comb[8], comb[9])
  cols = df.columns


  categoricalColumns = []
  numericCols = []
  for cmb in range(10):
    if df.dtypes[cmb+1][1] == 'string':
      categoricalColumns.append(cols[cmb+1])
    elif df.dtypes[cmb+1][1] == 'int':
      numericCols.append(cols[cmb+1])

  stages = []

  for categoricalCol in categoricalColumns:
      stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
      encoder = OneHotEncoderEstimator(inputCols= [stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
      stages += [stringIndexer, encoder]

  label_stringIdx = StringIndexer(inputCol = 'Accident_Severity', outputCol = 'label')
  stages += [label_stringIdx]

  assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
  assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
  stages += [assembler]


  pipeline = Pipeline(stages = stages)
  pipelineModel = pipeline.fit(df)
  df = pipelineModel.transform(df)
  selectedCols = ['label', 'features'] + cols
  df = df.select(selectedCols)

  train, test = df.randomSplit([0.8, 0.2])


  #Perceptron Algo
  layers = [10, 5, 6, 2]
  mlpc = MultilayerPerceptronClassifier(layers = layers, blockSize = 32, seed = 4, featuresCol = 'features', labelCol = 'label')
  model = mlpc.fit(train)
  # compute accuracy on the test set
  result = model.transform(test)
  filename = "/home/linuxu/Desktop/comb" + str(i) + "_Perceptron.csv"
  result.toPandas().to_csv(filename)

  resultsPerceptron = result.select(['prediction', 'label'])
  predictionAndLabels = resultsPerceptron.rdd
  metricsPerceptron = MulticlassMetrics(predictionAndLabels)

  fr = open("/home/linuxu/Desktop/results.txt","a+")
  fr.write("Column Combination: "+ ', '.join(map(str, comb)) + "\n")
  fr.write("Results for Perceptron Algorithm: \n")
  fr.write("Accuracy: " + str((metricsPerceptron.accuracy)*100) + "\n")
  fr.write("Precision: " + str(metricsPerceptron.weightedPrecision) + "\n")
  fr.write("Recall: " + str(metricsPerceptron.weightedRecall) + "\n")
  fr.write("TruePositiveRate: " + str(metricsPerceptron.weightedTruePositiveRate) + "\n")
  fr.write("FalsePositiveRate: " + str(np.nan_to_num(metricsPerceptron.weightedFalsePositiveRate)) + "\n\n\n\n")
  fr.close()

  print("Perceptron Algo done") 
    
  spark.catalog.clearCache()
  del comb
  del categoricalColumns
  del numericCols
  del stages
  print(i, "pass completed")
  i = i + 1

Perceptron Algo done
1 pass completed
